In [86]:
import socket
from struct import unpack, pack

In [87]:
def registerRequest_encode(nick, ip, port):
    return pack(f'BB{len(nick)}s{4}sH', 1, len(nick), nick.encode('utf-8'), socket.inet_aton(ip), port)

def addUser_encode(nick, ip, port):
    return pack(f'BB{len(nick)}s{4}sH', 3, len(nick), nick.encode('utf-8'), socket.inet_aton(ip), port)

def addUser_decode(data):
    return unpack(f'BB{len(data)-9}sBBBBH', data)

def broadcastMsg_encode(msg):
    return pack(f'BB{len(msg)}s', 5, len(msg), msg.encode('utf-8'))

def broadcast_encode(nick, msg):
    return pack(f'BB{len(nick)}sB{len(msg)}s', 6, len(nick), nick.encode('utf-8'), len(msg), msg.encode('utf-8'))

def logout_encode():
    return pack(f'B', 7)

def removeUser_encode(nick):
    return pack(f'BB{len(nick)}s', 8, len(nick), nick.encode('utf-8'))

In [88]:
def addUser_decode2(data):
        decoded = unpack(f'BB{len(data)-9}sBBBBH', data)
        msgtype = decoded[0]
        nick = decoded[2].decode('utf-8')
        ip = str(decoded[3])+'.'+str(decoded[4])+'.'+str(decoded[5])+'.'+str(decoded[6])
        port = decoded[7]
        return (msgtype, nick, ip, port)

In [93]:
def broadcastMsg_decode(data):
        decoded = unpack(f'BB{len(data)-2}s', data)
        return decoded[2].decode('utf-8')

In [94]:
msg = 'hallo welt'
encoded = broadcastMsg_encode(msg)
decoded = broadcastMsg_decode(encoded)
print(decoded)

hallo welt


In [89]:
ip = '127.0.0.2'
nick = 'max'
port = 50

In [90]:
userEncoded = addUser_encode(nick, ip, port)
userDecoded = addUser_decode2(userEncoded)
print(userDecoded)

(3, 'max', '127.0.0.2', 50)


In [69]:
print(socket.inet_aton(ip))
print(addUser_encode('max', '127.0.0.2', 50))
print(addUser_decode(addUser_encode('max', '127.0.0.2', 50)))

b'\x7f\x00\x00\x02'
b'\x03\x03max\x7f\x00\x00\x02\x002\x00'
(3, 3, b'max', 127, 0, 0, 2, 50)


In [71]:
data = pack(f'B{4}s', 1, '1234'.encode('utf-8'))
unpacked = unpack(f'B{4}s', data)
print(data[0])
print(unpacked[0])
print(unpacked[1].decode('utf-8'))

1
1
1234
